# 📦 Instalacja pakietu `xnat_dcm2bids`

In [82]:
!pip install --force-reinstall https://github.com/nencki-lobi/xnat_dcm2bids/archive/refs/heads/master.zip > /dev/null 2>&1

# 🔐 Konfiguracja połączenia z XNAT

Przejdź do okienka Terminala i wpisz `xnat-get`, żeby skonfigurować połączenie z XNAT.  

Powinno się pojawić `ERROR! project_id ("-p") must be provided to use empty IDs string`

# 📥 Pobranie listy sesji z projektu XNAT

In [80]:
%cd ~
!xnat_getcsv mj_ris.csv mj_ris

/home/jovyan
🟢 Data saved to mj_ris.csv


In [5]:
import pandas as pd
df = pd.read_csv("mj_ris.csv",dtype=str,index)
df.head(10)

,MR ID,Subject_ID,subject,session
0,2022_10_27-12_58_51-DST-1_3_12_2_1107_5_99_3,0064,0064,NaN
1,f490f566-2124-46,MJ_RIS_003,003,NaN
2,e5643cf8-6cc7-49,MJ_RIS_005_2,0052,NaN
3,cae9cfc8-5f84-47,MJ_RIS_006,006,NaN
4,50ab6ce9-29df-4b,MJ_RIS_006_T2,0062,NaN
5,803b1e6f-ff23-42,MJ_RIS_007,007,NaN
6,d6880ffc-5a0c-49,MJ_RIS_007_02,00702,NaN
7,6f2a0e25-0da2-49,MJ_RIS_008,008,NaN
8,bd4e995c-4d2f-40,MJ_RIS_008_02,00802,NaN
9,7f6f2520-3dfd-41,MJ_RIS_010,010,NaN


# 🏗️ Przygotowanie struktury katalogów BIDS

In [21]:
%cd ~
!dcm2bids_scaffold -o /home/jovyan/bids-dir

/home/jovyan
INFO    | --- dcm2bids_scaffold start ---
INFO    | Running the following command: /opt/conda/bin/dcm2bids_scaffold -o /home/jovyan/bids-dir
INFO    | OS version: Linux-5.15.0-143-generic-x86_64-with-glibc2.39
INFO    | Python version: 3.12.11 | packaged by conda-forge | (main, Jun  4 2025, 14:45:31) [GCC 13.3.0]
INFO    | dcm2bids version: 3.2.0
INFO    | Checking for software update
INFO    | Currently using the latest version of dcm2bids.
INFO    | The files used to create your BIDS directory were taken from https://github.com/bids-standard/bids-starter-kit. 

INFO    | Tree representation of /home/jovyan/bids-dir/
INFO    | /home/jovyan/bids-dir/
INFO    | ├── code/
INFO    | ├── derivatives/
INFO    | ├── sourcedata/
INFO    | ├── tmp_dcm2bids/
INFO    | │   └── log/
INFO    | │       └── scaffold_20250804-104446.log
INFO    | ├── .bidsignore
INFO    | ├── CHANGES
INFO    | ├── dataset_description.json
INFO    | ├── participants.json
INFO    | ├── participants.tsv
INF

In [22]:
%cd ~/bids-dir

/home/jovyan/bids-dir


In [24]:
!ls -1 ~/bids-dir

CHANGES
README
code
dataset_description.json
derivatives
participants.json
participants.tsv
sourcedata
tmp_dcm2bids


In [25]:
!ln -sf ~/lobi-mri-scripts/dcm2bids/mjris.json ./code/config.json

# ▶️ Przetworzenie jednej sesji

xnat session id: `f490f566-2124-46`  
subject: `03`  
session: `01`


In [31]:
!xnat_dcm2bids --bids-dir ~/bids-dir --config ./code/config.json f490f566-2124-46 03 01

🟢 Downloading f490f566-2124-46 to /home/jovyan/bids-dir/sourcedata
WARNING - Detected a redirect from http://xnat.nencki.edu.pl to https://xnat.nencki.edu.pl/, using https://xnat.nencki.edu.pl/ from now on
WARNING - Login using JSESSION failed, falling back to basic-auth.
INFO: All accessible sessions that matched pattern(s) 'f490f566-2124-46' were skipped:
f490f566-2124-46
🟢 Running dcm2bids for subject 03, session 01
INFO    | --- dcm2bids start ---
INFO    | Running the following command: /opt/conda/bin/dcm2bids --auto_extract_entities -p 03 -s 01 -c code/config.json -o /home/jovyan/bids-dir -d /home/jovyan/bids-dir/sourcedata/f490f566-2124-46
INFO    | OS version: Linux-5.15.0-143-generic-x86_64-with-glibc2.39
INFO    | Python version: 3.12.11 | packaged by conda-forge | (main, Jun  4 2025, 14:45:31) [GCC 13.3.0]
INFO    | dcm2bids version: 3.2.0
INFO    | dcm2niix version: v1.0.20250505
INFO    | Checking for software update
INFO    | Currently using the latest version of dcm2bids

# 🔁 Przetworzenie wielu sesji z CSV

In [37]:
!while IFS=',' read -r id _ sub ses _; do \
  echo xnat-dcm2bids --config ./code/config.json "$id" "$sub" 1; \
done < ~/mj_ris.csv

xnat-dcm2bids --config ./code/config.json MR ID subject
xnat-dcm2bids --config ./code/config.json 2022_10_27-12_58_51-DST-1_3_12_2_1107_5_99_3 0064
xnat-dcm2bids --config ./code/config.json f490f566-2124-46 003
xnat-dcm2bids --config ./code/config.json e5643cf8-6cc7-49 0052
xnat-dcm2bids --config ./code/config.json cae9cfc8-5f84-47 006
xnat-dcm2bids --config ./code/config.json 50ab6ce9-29df-4b 0062
xnat-dcm2bids --config ./code/config.json 803b1e6f-ff23-42 007
xnat-dcm2bids --config ./code/config.json d6880ffc-5a0c-49 00702
xnat-dcm2bids --config ./code/config.json 6f2a0e25-0da2-49 008
xnat-dcm2bids --config ./code/config.json bd4e995c-4d2f-40 00802
xnat-dcm2bids --config ./code/config.json 7f6f2520-3dfd-41 010
xnat-dcm2bids --config ./code/config.json 79672b55-27b4-45 011
xnat-dcm2bids --config ./code/config.json e5276683-0807-43 012
xnat-dcm2bids --config ./code/config.json d5ece856-31fc-4d 01202
xnat-dcm2bids --config ./code/config.json cea857a5-118b-44 013
xnat-dcm2bids --config ./

# 🔁 Uruchomienie skryptów mriqc, fmriprep etc.

Uwaga! Wymagane jest środowisko neurodesk (Calcus, Mikołajki ew. neurodesktop).

In [51]:
!lobi_script run_mriqc.sh 03 ~/bids-dir ~/bids-dir/derivatives/mriqc

▶️  Running: bash -l -c "/home/jovyan/lobi-mri-scripts/run_mriqc.sh 03 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/mriqc"
/home/jovyan/mriqc
------------------------------------------------------------------
  Running MRIQC version 24.1.0.dev0+gd5b13cb5.d20240826
  ----------------------------------------------------------------

  NOTICE
  Copyright © The NiPreps Developers.
  
  This product includes software developed by
  the NiPreps Community (https://nipreps.org/).
  
  Portions of this software were developed at the Department of
  Psychology at Stanford University, Stanford, CA, US.
  
  This software contains code ultimately derived from the
  PCP Quality Assessment Protocol (QAP;
  http://preprocessed-connectomes-project.org/quality-assessment-protocol)
  by C. Craddock, S. Giavasis, D. Clark, Z. Shezhad, and J. Pellman.
  
  This software is also distributed as a Docker container image.
  The bootstrapping file for the image ("Dockerfile") is licensed
  under the

In [81]:
!tail -n +2 ~/mj_ris.csv | parallel --colsep ',' echo lobi_script run_fmriprep_min.sh {{3}} ~/bids-dir ~/bids-dir/derivatives/fmriprep


lobi_script run_fmriprep_min.sh 0064 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/fmriprep
lobi_script run_fmriprep_min.sh 003 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/fmriprep
lobi_script run_fmriprep_min.sh 0052 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/fmriprep
lobi_script run_fmriprep_min.sh 006 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/fmriprep
lobi_script run_fmriprep_min.sh 0062 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/fmriprep
lobi_script run_fmriprep_min.sh 007 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/fmriprep
lobi_script run_fmriprep_min.sh 00702 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/fmriprep
lobi_script run_fmriprep_min.sh 008 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/fmriprep
lobi_script run_fmriprep_min.sh 00802 /home/jovyan/bids-dir /home/jovyan/bids-dir/derivatives/fmriprep
lobi_script run_fmriprep_min.sh 010 /home/jovyan/bids-dir /home/jovyan/bids-dir/deri